In [1]:
import ultralytics 

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\masas\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import torch
torch.__version__

'2.5.1+cpu'

Detect and Track and Count people

In [7]:
%cd yolov8_DeepSORT

[WinError 2] The system cannot find the file specified: 'yolov8_DeepSORT'
d:\personal_projects\computer_vision\yolov8_deepsort


d:\personal_projects\computer_vision\yolov8_deepsort\myenv\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [8]:
from ultralytics import YOLO
import time
import torch
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

In [13]:
model = YOLO("yolov8n.pt")

results = model("images/uav.jpg", save = True)

class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    probs = result.probs  # Class probabilities for classification outputs
    cls = boxes.cls.tolist()  # Convert tensor to list
    xyxy = boxes.xyxy
    xywh = boxes.xywh  # box with xywh format, (N, 4)
    conf = boxes.conf
    print(cls)
    for class_index in cls:
        class_name = class_names[int(class_index)]
        print("Class:", class_name)


image 1/1 d:\personal_projects\computer_vision\yolov8_deepsort\images\uav.jpg: 448x640 1 airplane, 83.9ms
Speed: 0.0ms preprocess, 83.9ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict2
[4.0]
Class: airplane


DeepSORT

In [15]:
from deep.deep_sort.utils.parser import get_config
from deep.deep_sort.deep_sort import DeepSort
from deep.deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep/deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [16]:
import cv2 as cv2
video_path = 'images/uav_vid.mp4'

cap = cv2.VideoCapture(video_path)

# get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_hight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print(fps)

30.0


In [18]:
# Define the codec and create VideoWritter object

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path,fourcc, fps,(frame_width,frame_hight))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

frames = []

unique_track_ids = set()

In [ ]:

i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():

    ret, frame = cap.read()
    if ret:
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO('yolov8n.pt')

        results = model(frame, classes=4, conf=0.8)

        for result in results:
            boxes = result.boxes
            probs = result.probs
            cls = boxes.cls.tolist()
            xyxy = boxes.xyxy
            xywh = boxes.xywh
            for class_idnex in cls:
                class_name = class_names[int(class_idnex)]
        
        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()
            w = x2 -x1
            h = y2 - y1

            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int( x1 +w) , int(y1+ h)), color, 2)

            text_color = (0,0,0)
            cv2.putText(og_frame, f"{class_name}--{track_id}",(int(x1) +10, int(y1) -5 ), cv2.FONT_HERSHEY_SIMPLEX, text_color,1, cv2.LINE_AA)

            unique_track_ids.add(track_id)

        uav_count = len(unique_track_ids)

        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter +=1

        if elapsed >1:
            fps = counter/elapsed
            counter = 0
            start_time = current_time

        cv2.putText(og_frame, f"uav count : {uav_count}" , (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (225,225,255), 2, cv2.LINE_AA)

        frames.append(og_frame)

        out.write(cv2.cvtColor(og_frame,cv2.COLOR_RGB2BGR))

cap.release()
out.release()
cv2.destroyAllWindows()



0: 352x640 1 airplane, 72.6ms
Speed: 0.0ms preprocess, 72.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)


AttributeError: 'numpy.ndarray' object has no attribute 'detach'